In [45]:
import requests
import pandas as pd


### 1. 인터넷에서 데이터 가져오기

In [46]:
# 지니차트 top200

url = 'https://genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [47]:
# Chrome User agent

header = {'User-Agent':
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
}
req = requests.get(url, headers=header)
html = req.text
# html

In [48]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')


### 찾으려고 하는 데이터의 노드 찾기

In [49]:
trs = soup.find_all('tr')
len(trs)

51

In [50]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)


50

In [51]:
# 우클릭 > copy selector

tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

In [52]:
# 우클릭 > copy Xpath  (Selenium에서 사용 가능 / BeautifulSoup은 X)
 
# //*[@id="body-content"]/div[6]/div/table/tbody    

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출
- rank, title, artist, album

In [53]:
tr = trs[0]

In [54]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [55]:
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'1'

In [56]:
#title

tr.select_one('.title.ellipsis').get_text()       # 클래스가 두개(스페이스로 구분)

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                리무진 (Feat. MINO) (Prod. by GRAY)'

In [57]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'리무진 (Feat. MINO) (Prod. by GRAY)'

In [58]:
# artist

artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

"BE'O (비오)"

In [59]:
# album

album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'쇼미더머니 10 Episode 3'

### 4. 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데이터프레임 만들기

- 1) 개별리스트 만들기 (reank_list, title_list...)
- 2) 딕셔너리 리스트 만들기 {순위:rank, 제목:title...}

In [60]:
# 개별리스트 만들기

rank_list , title_list, artist_list, album_list = [],[],[],[]

for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    rank_list.append(rank), 
    title_list.append(title), 
    artist_list.append(artist), 
    album_list.append(album)

In [61]:
df = pd.DataFrame({
    '순위':rank_list,'제목':title_list,'가수':artist_list,'앨범':album_list
})
df.set_index('순위', inplace=True)
df.head()

,제목,가수,앨범
순위,,,
1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [62]:
# 방법 2

lines = []

for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank,title,artist,album])

df = pd.DataFrame(lines, columns=['순위','제목','가수','앨범'])
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [63]:
# 방법3

lines = []

for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    d={'순위':rank,'제목':title,'가수':artist,'앨범':album}
    lines.append(d)

df = pd.DataFrame(lines)
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


### 5. 모든페이지의 데이터 가져오기

In [66]:
lines = []

for page in range(1,5):
    url = f'https://genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')


    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        lines.append([rank,title,artist,album])

df = pd.DataFrame(lines, columns=['순위','제목','가수','앨범'])
df.head()
    

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [67]:
df.to_csv('지니차트_Top200(211206).csv', index=False)